In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\HARSH\Desktop\python dataset\emotion_nlp.csv")

In [4]:
df.head()

,Text,Emotions
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
df.shape

(17999, 2)

In [6]:
df.Emotions.value_counts()

joy         6056
sadness     5247
anger       2434
fear        2161
love        1463
surprise     638
Name: Emotions, dtype: int64

In [7]:
df.isnull().sum()

Text        0
Emotions    0
dtype: int64

In [8]:
df = df.rename(columns = ({"Emotions" : "Y" , "Text" : "X"}))
df.head()

,X,Y
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [9]:
df.Y.value_counts()

joy         6056
sadness     5247
anger       2434
fear        2161
love        1463
surprise     638
Name: Y, dtype: int64

In [16]:
df.Y.unique()

array(['sadness', 'anger', 'love', 'surprise', 'fear', 'joy'],
      dtype=object)

In [17]:
df.Y.replace({'sadness':0, 'anger':1, 'love':2, 'surprise':3, 'fear':4, 'joy':5},inplace=True)

In [19]:
df.X = df.X.str.lower() # Mandtory # # convert all text to lower case mandatory step
df

,X,Y
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
17994,i just keep feeling like someone is being unki...,1
17995,im feeling a little cranky negative after this...,1
17996,i feel that i am useful to my people and that ...,5
17997,im feeling more comfortable with derby i feel ...,5


In [11]:
import nltk # nltk - natural language toolkit

In [12]:
from nltk.corpus import stopwords

In [13]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
l1 = stopwords.words("english")

In [15]:
def text_process(mess):            ### creating a function

    """                                                        ## a docstring

    1. remove the punctuation

    2. remove the stopwords

    3. return the list of clean textwords

    

    """

    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = "".join(nopunc)

    

    return [ word for word in nopunc.split() if word not in l1]

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
bow_transformer = CountVectorizer(analyzer = text_process).fit(df["X"])

In [22]:
bow_transformer.vocabulary_

{'didnt': 3834,
 'feel': 5205,
 'humiliated': 6786,
 'go': 5997,
 'feeling': 5209,
 'hopeless': 6687,
 'damned': 3392,
 'hopeful': 6685,
 'around': 726,
 'someone': 13096,
 'cares': 2062,
 'awake': 974,
 'im': 6920,
 'grabbing': 6058,
 'minute': 8934,
 'post': 10731,
 'greedy': 6120,
 'wrong': 15877,
 'ever': 4830,
 'nostalgic': 9576,
 'fireplace': 5342,
 'know': 7832,
 'still': 13463,
 'property': 11032,
 'grouchy': 6160,
 'ive': 7442,
 'little': 8241,
 'burdened': 1870,
 'lately': 7950,
 'wasnt': 15482,
 'sure': 13811,
 'taking': 13975,
 'milligrams': 8890,
 'times': 14361,
 'recommended': 11455,
 'amount': 484,
 'fallen': 5093,
 'asleep': 798,
 'lot': 8351,
 'faster': 5140,
 'also': 429,
 'like': 8164,
 'funny': 5747,
 'confused': 2837,
 'life': 8137,
 'teenager': 14091,
 'jaded': 7458,
 'year': 15934,
 'old': 9767,
 'man': 8527,
 'petronas': 10378,
 'years': 15942,
 'performed': 10304,
 'well': 15573,
 'made': 8463,
 'huge': 6763,
 'profit': 10985,
 'romantic': 11990,
 'make': 8501

In [23]:
# Create a TDM

In [24]:
tdm = bow_transformer.transform(df.X)

In [25]:
tdm.shape

(17999, 16035)

In [26]:
# now TDM will act as X in sampling stage :--

#### Sampling stage:--

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
train_X  , test_X , train_Y  , test_Y = train_test_split(tdm, df.Y , test_size = .20)

#### model building stage:--

####  NAIVE BAYES

In [29]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_X , train_Y)

MultinomialNB()

In [31]:
nb_pred = nb.predict(test_X)
nb_pred

array([0, 0, 5, ..., 4, 0, 5], dtype=int64)

In [32]:
from sklearn.metrics import *

In [33]:
tab_nb = confusion_matrix(test_Y , nb_pred)
tab_nb = pd.DataFrame(tab_nb)
tab_nb.columns = ['sadness','anger','love','surprise','fear','joy']
tab_nb.index = ['sadness','anger','love','surprise','fear','joy']
tab_nb

,sadness,anger,love,surprise,fear,joy
sadness,943,8,2,1,4,34
anger,87,328,0,0,12,54
love,42,2,90,0,3,150
surprise,45,2,0,14,26,43
fear,91,22,1,0,256,56
joy,61,7,6,1,9,1200


In [34]:
print(classification_report(test_Y , nb_pred))

              precision    recall  f1-score   support

           0       0.74      0.95      0.83       992
           1       0.89      0.68      0.77       481
           2       0.91      0.31      0.47       287
           3       0.88      0.11      0.19       130
           4       0.83      0.60      0.70       426
           5       0.78      0.93      0.85      1284

    accuracy                           0.79      3600
   macro avg       0.84      0.60      0.64      3600
weighted avg       0.80      0.79      0.76      3600



#### Logistic Regression Model :--

In [35]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(train_X , train_Y)
pred_logreg = logreg.predict(test_X)

C:\Users\HARSH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
tab_logreg = confusion_matrix(test_Y , pred_logreg)
tab_logreg = pd.DataFrame(tab_logreg)
tab_logreg.columns = ['sadness','anger','love','surprise','fear','joy']
tab_logreg.index = ['sadness','anger','love','surprise','fear','joy']
tab_logreg

,sadness,anger,love,surprise,fear,joy
sadness,912,29,9,4,18,20
anger,16,430,3,0,18,14
love,1,4,260,0,0,22
surprise,1,0,2,103,20,4
fear,16,20,3,24,356,7
joy,19,8,71,18,12,1156


In [37]:
print(classification_report(test_Y , pred_logreg))

              precision    recall  f1-score   support

           0       0.95      0.92      0.93       992
           1       0.88      0.89      0.88       481
           2       0.75      0.91      0.82       287
           3       0.69      0.79      0.74       130
           4       0.84      0.84      0.84       426
           5       0.95      0.90      0.92      1284

    accuracy                           0.89      3600
   macro avg       0.84      0.87      0.86      3600
weighted avg       0.90      0.89      0.90      3600



#### Building Decision Tree :--

In [51]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()

In [52]:
dt.fit(train_X ,train_Y)

DecisionTreeClassifier()

In [53]:
pred_dt = dt.predict(test_X)

In [54]:
tab_dt = confusion_matrix(test_Y , pred_dt)
tab_dt= pd.DataFrame(tab_dt)
tab_dt.columns = ['sadness','anger','love','surprise','fear','joy']
tab_dt.index = ['sadness','anger','love','surprise','fear','joy']
tab_dt

,sadness,anger,love,surprise,fear,joy
sadness,910,34,9,11,17,11
anger,25,411,3,1,28,13
love,2,4,223,1,2,55
surprise,3,0,0,104,16,7
fear,22,26,2,25,345,6
joy,48,16,58,13,17,1132


In [55]:
print(classification_report(test_Y , pred_dt))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       992
           1       0.84      0.85      0.85       481
           2       0.76      0.78      0.77       287
           3       0.67      0.80      0.73       130
           4       0.81      0.81      0.81       426
           5       0.92      0.88      0.90      1284

    accuracy                           0.87      3600
   macro avg       0.82      0.84      0.83      3600
weighted avg       0.87      0.87      0.87      3600



#### Adaboost:-

In [56]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(dt)

In [58]:
abc.fit(train_X,train_Y)
pred_abc = abc.predict(test_X)

In [59]:
tab_abc = confusion_matrix(test_Y , pred_abc)
tab_abc= pd.DataFrame(tab_abc)
tab_abc.columns = ['sadness','anger','love','surprise','fear','joy']
tab_abc.index = ['sadness','anger','love','surprise','fear','joy']
tab_abc

,sadness,anger,love,surprise,fear,joy
sadness,881,23,6,6,13,63
anger,50,371,2,0,18,40
love,22,7,172,0,3,83
surprise,8,1,0,90,21,10
fear,32,23,1,21,321,28
joy,78,10,27,12,13,1144


In [60]:
print(classification_report(test_Y , pred_abc))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       992
           1       0.85      0.77      0.81       481
           2       0.83      0.60      0.69       287
           3       0.70      0.69      0.69       130
           4       0.83      0.75      0.79       426
           5       0.84      0.89      0.86      1284

    accuracy                           0.83      3600
   macro avg       0.81      0.77      0.78      3600
weighted avg       0.83      0.83      0.83      3600



#### XGBoost Classifier :--

In [61]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(train_X , train_Y)
pred_xgb = xgb.predict(test_X)

In [62]:
tab_xgb = confusion_matrix(test_Y , pred_xgb)
tab_xgb = pd.DataFrame(tab_xgb )
tab_xgb .columns = ['sadness','anger','love','surprise','fear','joy']
tab_xgb .index = ['sadness','anger','love','surprise','fear','joy']
tab_xgb 

,sadness,anger,love,surprise,fear,joy
sadness,897,21,10,8,14,42
anger,16,412,1,0,16,36
love,2,4,239,0,0,42
surprise,1,0,0,95,27,7
fear,8,12,2,11,376,17
joy,15,6,67,17,14,1165


In [63]:
print(classification_report(test_Y , pred_xgb))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93       992
           1       0.91      0.86      0.88       481
           2       0.75      0.83      0.79       287
           3       0.73      0.73      0.73       130
           4       0.84      0.88      0.86       426
           5       0.89      0.91      0.90      1284

    accuracy                           0.88      3600
   macro avg       0.84      0.85      0.85      3600
weighted avg       0.89      0.88      0.89      3600



#### Random Forest:---

In [64]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(class_weight='balanced')

In [65]:
rfc.fit(train_X ,train_Y)

RandomForestClassifier(class_weight='balanced')

In [67]:
pred_rfc = rfc.predict(test_X)

In [68]:
tab_rfc = confusion_matrix(test_Y , pred_rfc)
tab_rfc= pd.DataFrame(tab_rfc )
tab_rfc .columns = ['sadness','anger','love','surprise','fear','joy']
tab_rfc .index = ['sadness','anger','love','surprise','fear','joy']
tab_rfc

,sadness,anger,love,surprise,fear,joy
sadness,895,34,11,8,21,23
anger,16,430,0,1,13,21
love,0,3,236,0,1,47
surprise,3,0,0,105,17,5
fear,9,25,3,28,355,6
joy,28,11,59,13,16,1157


In [69]:
print(classification_report(test_Y , pred_rfc))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92       992
           1       0.85      0.89      0.87       481
           2       0.76      0.82      0.79       287
           3       0.68      0.81      0.74       130
           4       0.84      0.83      0.84       426
           5       0.92      0.90      0.91      1284

    accuracy                           0.88      3600
   macro avg       0.83      0.86      0.85      3600
weighted avg       0.89      0.88      0.88      3600



In [74]:
df.shape

(17999, 2)

In [75]:
np.sqrt(17999)

134.1603518182626

#### KNN :--

In [80]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(train_X,train_Y)

KNeighborsClassifier()

In [81]:
pred_knn = knn.predict(test_X)

In [82]:
tab_knn = confusion_matrix(test_Y , pred_knn)
tab_knn= pd.DataFrame(tab_knn)
tab_knn .columns = ['sadness','anger','love','surprise','fear','joy']
tab_knn .index = ['sadness','anger','love','surprise','fear','joy']
tab_knn

,sadness,anger,love,surprise,fear,joy
sadness,793,47,6,1,25,120
anger,156,232,6,1,17,69
love,104,22,73,2,10,76
surprise,46,10,1,29,18,26
fear,147,48,4,15,150,62
joy,386,47,42,7,40,762


In [83]:
print(classification_report(test_Y , pred_knn))

              precision    recall  f1-score   support

           0       0.49      0.80      0.60       992
           1       0.57      0.48      0.52       481
           2       0.55      0.25      0.35       287
           3       0.53      0.22      0.31       130
           4       0.58      0.35      0.44       426
           5       0.68      0.59      0.64      1284

    accuracy                           0.57      3600
   macro avg       0.57      0.45      0.48      3600
weighted avg       0.59      0.57      0.55      3600



#### SVM :-

In [104]:
from sklearn.svm import SVC
svc= SVC(class_weight='balanced',kernel='poly', degree=3, coef0=1)
svc.fit(train_X ,train_Y)
pred_svc = svc.predict(test_X)

In [105]:
tab_svc = confusion_matrix(test_Y , pred_svc)
tab_svc= pd.DataFrame(tab_svc)
tab_svc .columns = ['sadness','anger','love','surprise','fear','joy']
tab_svc.index = ['sadness','anger','love','surprise','fear','joy']
tab_svc

,sadness,anger,love,surprise,fear,joy
sadness,893,35,14,8,21,21
anger,16,427,4,1,17,16
love,0,3,247,0,1,36
surprise,2,0,2,100,23,3
fear,14,28,6,22,352,4
joy,32,10,73,20,14,1135


In [106]:
print(classification_report(test_Y , pred_svc))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92       992
           1       0.85      0.89      0.87       481
           2       0.71      0.86      0.78       287
           3       0.66      0.77      0.71       130
           4       0.82      0.83      0.82       426
           5       0.93      0.88      0.91      1284

    accuracy                           0.88      3600
   macro avg       0.82      0.85      0.83      3600
weighted avg       0.88      0.88      0.88      3600

